In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加载MNIST数据集
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')
X = X.astype(float)
y = y.astype(int)

# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转换为numpy数组以避免索引问题
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# 标准化数据
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 添加偏置项
X_train_bias = np.hstack((X_train, np.ones((X_train.shape[0], 1))))
X_test_bias = np.hstack((X_test, np.ones((X_test.shape[0], 1))))

In [ ]:
class BinaryPerceptron:
    def __init__(self, n_features):
        self.weights = np.zeros(n_features)
    
    def predict(self, X):
        return np.sign(np.dot(X, self.weights))
    
    def train(self, X, y, epochs=10):
        n_samples = X.shape[0]
        
        for epoch in range(epochs):
            errors = 0
            
            for i in range(n_samples):
                x_i = X[i]
                y_i = y[i]
                
                if y_i * np.dot(self.weights, x_i) <= 0:  # 如果分类错误
                    self.weights += y_i * x_i
                    errors += 1
            
            error_rate = errors / n_samples
            if epoch == epochs - 1:
                print(f"Final epoch error: {error_rate:.4f}")

class OneVsAllPerceptron:
    def __init__(self, n_classes, n_features, epochs=10):
        self.n_classes = n_classes
        self.epochs = epochs
        self.classifiers = [BinaryPerceptron(n_features) for _ in range(n_classes)]
    
    def train(self, X, y):
        for epoch in range(self.epochs):
            print(f"Epoch {epoch+1}/{self.epochs}")
            
            for k in range(self.n_classes):
                # 为当前类别创建二元标签
                binary_y = np.where(y == k, 1, -1)
                
                # 训练当前类别的二元感知机
                self.classifiers[k].train(X, binary_y, epochs=1)
    
    def predict(self, X):
        # 对每个样本，计算所有类别的分数
        scores = np.zeros((X.shape[0], self.n_classes))
        
        for k in range(self.n_classes):
            # 计算第k个分类器对每个样本的分数
            scores[:, k] = np.dot(X, self.classifiers[k].weights)
        
        # 返回得分最高的类别
        return np.argmax(scores, axis=1)
    
    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return 1 - accuracy  # 返回错误率

In [ ]:
class MulticlassPerceptron:
    def __init__(self, n_classes, n_features, epochs=10):
        self.n_classes = n_classes
        self.epochs = epochs
        self.weights = np.zeros((n_classes, n_features))
        
    def train(self, X, y):
        n_samples = X.shape[0]
        
        for epoch in range(self.epochs):
            errors = 0
            
            for i in range(n_samples):
                x_i = X[i]
                y_i = y[i]
                
                # 计算每个类别的得分
                scores = np.dot(self.weights, x_i)
                
                # 找出得分最高的类别
                predicted_class = np.argmax(scores)
                
                # 如果预测错误，更新权重
                if predicted_class != y_i:
                    errors += 1
                    self.weights[y_i] += x_i  # 增加正确类别的权重
                    self.weights[predicted_class] -= x_i  # 减少错误类别的权重
            
            error_rate = errors / n_samples
            print(f"Epoch {epoch+1}/{self.epochs}, Training Error: {error_rate:.4f}")
    
    def predict(self, X):
        # 对每个样本，计算所有类别的分数
        scores = np.array([np.dot(self.weights, x_i) for x_i in X])
        
        # 返回得分最高的类别
        return np.argmax(scores, axis=1)
    
    def evaluate(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return 1 - accuracy  # 返回错误率

In [3]:
# 训练One vs. All感知机
print("Training One vs. All Perceptron:")
num_classes = 10  # MNIST有10个类别
ova_perceptron = OneVsAllPerceptron(num_classes, X_train_bias.shape[1], epochs=10)
ova_perceptron.train(X_train_bias, y_train)

# 评估模型
train_error_ova = ova_perceptron.evaluate(X_train_bias, y_train)
test_error_ova = ova_perceptron.evaluate(X_test_bias, y_test)

print(f"Final Training Error (One vs. All): {train_error_ova:.4f}")
print(f"Final Test Error (One vs. All): {test_error_ova:.4f}")


Training One vs. All Perceptron:


NameError: name 'OneVsAllPerceptron' is not defined

In [ ]:
# 训练多类感知机
print("\nTraining Multiclass Perceptron:")
multi_perceptron = MulticlassPerceptron(num_classes, X_train_bias.shape[1], epochs=10)
multi_perceptron.train(X_train_bias, y_train)

# 评估模型
train_error_multi = multi_perceptron.evaluate(X_train_bias, y_train)
test_error_multi = multi_perceptron.evaluate(X_test_bias, y_test)

print(f"Final Training Error (Multiclass): {train_error_multi:.4f}")
print(f"Final Test Error (Multiclass): {test_error_multi:.4f}")

In [ ]:
# 绘制训练和测试错误率对比图
labels = ['One vs. All', 'Multiclass']
train_errors = [train_error_ova, train_error_multi]
test_errors = [test_error_ova, test_error_multi]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, train_errors, width, label='Training Error')
rects2 = ax.bar(x + width/2, test_errors, width, label='Test Error')

ax.set_ylabel('Error Rate')
ax.set_title('Error Rates by Model Type')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()
plt.show()